In [2]:
!pip install pytorch_lightning

/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
     |████████████████████████████████| 826 kB 17.5 MB/s eta 0:00:01
     |████████████████████████████████| 269 kB 58.1 MB/s eta 0:00:01
     |████████████████████████████████| 176 kB 42.6 MB/s eta 0:00:01
     |████████████████████████████████| 10.6 MB 50.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 62.8 MB/s eta 0:00:01
     |████████████████████████████████| 136 kB 54.0 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 670 kB/s  eta 0:00:01
     |████████████████████████████████| 129 kB 53.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.1 MB 49.6 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 57.3 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 59.3 MB/s e

In [38]:
import os
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split,TensorDataset
from torch.nn.utils import rnn
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split


def elementwise_apply(fn, *args):
    """A hack to apply fn like nn.Embedding, F.log_softmax to PackedSequence"""
    return rnn.PackedSequence(
        fn(*[(arg.data if type(arg) == rnn.PackedSequence else arg)
             for arg in args]), args[0].batch_sizes)


class EncoderRNN(pl.LightningModule):

    def __init__(self, hidden_size, embed_size, max_size):
        super(EncoderRNN, self).__init__()
        self.embedding = nn.Embedding(embed_size, hidden_size)
        self.prev_hidden = torch.zeros(1, max_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, inp):
        # print (self.embedding(inp).shape)
        # embedded = self.embedding(inp).view(1, 1, -1)
        embedded = self.embedding(inp)
        output = embedded
        output, self.prev_hidden = self.gru(output, self.prev_hidden)
        return output
    
class DecoderRNN(pl.LightningModule):

    def __init__(self, hidden_size, output_size, max_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.prev_hidden = torch.zeros(1, max_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, inp):
        # output = self.embedding(inp).view(1, 1, -1)
        output = self.embedding(inp)
        output = F.relu(output)
        output, self.prev_hidden = self.gru(output, self.prev_hidden)
        output = self.softmax(self.out(output[0]))
        return output



In [41]:
class Seq2Seq(pl.LightningModule):
    def __init__(self):
        super().__init__()
        n_tokens = 128
        hid_size = 12
        max_size = 16
        self.encoder = EncoderRNN(hid_size, n_tokens, max_size)
        self.decoder = DecoderRNN(hid_size, n_tokens, max_size)

    def forward(self, enc_out):
        # in lightning, forward defines the prediction/inference actions
        out = self.encoder(enc_out)
        return out

    def training_step(self, batch, batch_idx):
        # training_step defined the train loop. It is independent of forward
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.forward(x)
        x_hat = self.decoder(z)
        loss = nn.NLLLoss(x_hat, x)
        #self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


In [42]:
X = np.load("dummy_X.npy")
Y = np.load("dummy_Y.npy")
print (X.shape)
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=195)

tensor_x = torch.from_numpy(X)
tensor_y = torch.from_numpy(Y)

dataset = TensorDataset(tensor_x,tensor_y) # create your datset
# my_dataloader = DataLoader(my_dataset) # create your dataloader
train, val = random_split(dataset, [9000, 1000])

seq2seq = Seq2Seq()
trainer = pl.Trainer()
trainer.fit(seq2seq, DataLoader(train), DataLoader(val))

GPU available: True, used: False
TPU available: None, using: 0 TPU cores

  | Name    | Type       | Params
---------------------------------------
0 | encoder | EncoderRNN | 2.5 K 
1 | decoder | DecoderRNN | 4.1 K 
---------------------------------------
6.6 K     Trainable params
0         Non-trainable params
6.6 K     Total params
0.026     Total estimated model params size (MB)


(10000, 16)


Training: 0it [00:00, ?it/s]

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [30]:
16*12

192